## TFRecords creation
The goal of this Notebook is to **convert all the images that have been preprocessed in the Notebook 2_MRI_preprocessing and saved as a numpy’s compressed format (.npz) in the folder 'Datasets/Image_files' to TFRecords.**

According to [Tensorflow](https://www.tensorflow.org/tutorials/load_data/tfrecord), TFRecords are used to store the data as a sequence of binary strings. The main advantage of using TFRecords is that it speeds up data reading.

This notebook is structured as follows:
   - Initial set-up
   - Import libraries
   - Load features
   - Load labels
   - Create TFRecords

### Initial set-up

#### Google Colab

In [1]:
# Specify if user is working on Google Drive
google_colab = True

In [2]:
if google_colab == True:
    
    from google.colab import drive 
    drive.mount('/content/drive')
    
    path = './drive/MyDrive/TFM/Code/'
    
    import os
    os.chdir(path)

else:
    path = '../'
    
    import os
    os.chdir(path)

Mounted at /content/drive


### Import libraries

In [3]:
import os
import random
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tqdm
import tensorflow as tf
import glob

### Load features

#### Load 3D images directories (features)

In [6]:
# Specify folder where there are the 3D images in numpy’s compressed format (.npz)
directory_images = './Datasets/Image_files/'

In [7]:
# Get list of filenames from 3D volumes
filenames = os.listdir(directory_images)

# Remove ".DS_Store" file from list
if '.DS_Store' in filenames:
    filenames.remove('.DS_Store')

# Include all the path for each file
filenames = [directory_images + file for file in filenames]

# Check number of images loaded
print('[+] Number of 3D images:', len(filenames))

[+] Number of 3D images: 1146


In [8]:
# Shuffle list of filenames
random.shuffle(filenames)

#### Split dataset into training, validation and testing

In [ ]:
train_size = 0.7
test_size = 0.3

In [ ]:
train_filenames, test_filenames = train_test_split(filenames, 
                                               test_size = 0.3, 
                                               random_state = 7)

In [ ]:
test_filenames, val_filenames = train_test_split(test_filenames, 
                                             test_size = 0.5, 
                                             random_state = 7)

In [ ]:
# Check size of each dataset
print(f'[+] Training size:', len(train_filenames))
print(f'[+] Validation size:', len(val_filenames))
print(f'[+] Testing size:', len(test_filenames))

[+] Training size: 802
[+] Validation size: 172
[+] Testing size: 172


#### Load array for each 3D image (features)

In [ ]:
def return_volumes(filenames):
    '''
    Function used to load numpy arrays (.npz) from a list of directories
    Input: directories where there are the numpy arrays
    Output: array with 3D images
    ''' 

    # List where to save the 3D images
    volumes = []

    # Load 3D images from filenames list
    for file in filenames:

        # Read 3D image
        volume = np.load(file, allow_pickle= True)['arr_0']

        # Append 3D image to volumes list 
        volumes.append(volume)
        
        if len(volumes) % 100 == 0:
            print('[+] Number of images loaded:', len(volumes))
    
    print('Total number of images loaded:', len(volumes))
        
    return np.array(volumes)

In [ ]:
# Load training images
train_dataset = return_volumes(train_filenames)

[+] Number of images loaded: 100
[+] Number of images loaded: 200
[+] Number of images loaded: 300
[+] Number of images loaded: 400
[+] Number of images loaded: 500
[+] Number of images loaded: 600
[+] Number of images loaded: 700
[+] Number of images loaded: 800
Total number of images loaded: 802


In [ ]:
# Load validation images
val_dataset = return_volumes(val_filenames)

[+] Number of images loaded: 100
Total number of images loaded: 172


In [ ]:
# Load testing images
test_dataset = return_volumes(test_filenames)

[+] Number of images loaded: 100
Total number of images loaded: 172


In [ ]:
# Check size of each dataset
print(f'[+] Training shape:', train_dataset.shape)
print(f'[+] Validation shape:', val_dataset.shape)
print(f'[+] Testing shape:', test_dataset.shape)

[+] Training shape: (802, 110, 130, 80)
[+] Validation shape: (172, 110, 130, 80)
[+] Testing shape: (172, 110, 130, 80)


### Load labels

#### Load CSV files with image details: images IDs and class

In [10]:
# Load individuals CSV files with image details
df_1 = pd.read_csv('./Datasets/ADNI1_Complete_1Yr_1.5T.csv')
df_2 = pd.read_csv('./Datasets/ADNI1_Complete_2Yr_1.5T.csv')
df_3 = pd.read_csv('./Datasets/ADNI1_Complete_3Yr_1.5T.csv')

# Concatenate all CSV files in a unique dataframe
df = pd.concat([df_1, df_2, df_3])

# Remove extra whitespaces from column names
df.columns = df.columns.str.replace(" ", "")

df.head()

,ImageDataID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,AcqDate,Format,Downloaded
0,I125941,137_S_1426,MCI,M,85,4,MRI,MPR-R; GradWarp; N3; Scaled,Processed,10/30/2008,NiFTI,NaN
1,I121703,128_S_1408,MCI,M,73,4,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,9/19/2008,NiFTI,NaN
2,I121637,037_S_1421,MCI,F,76,4,MRI,MPR; GradWarp; N3; Scaled,Processed,9/17/2008,NiFTI,NaN
3,I122382,128_S_1407,MCI,F,76,4,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,9/05/2008,NiFTI,NaN
4,I121689,127_S_1427,MCI,F,71,4,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,9/02/2008,NiFTI,NaN


In [ ]:
# Retrieve only the image ID and Group (class) columns
df = df[['ImageDataID', 'Group']]
df.head()

,ImageDataID,Group
0,I125941,MCI
1,I121703,MCI
2,I121637,MCI
3,I122382,MCI
4,I121689,MCI


#### Load class for each 3D volume (labels)

In [ ]:
def return_labels(filenames, df):
    '''
    Function used to retrieve label for each sample of a dataset
    Input: directories where there are the images
    Output: array with labels
    ''' 

    # List where to save the class for each image: 0 (CN), 1 (AD)
    labels = []

    # Load labels from filenames list
    for file in filenames:

        # Get image ID from file name
        image_id = file.split('/')[-1].split('.')[0]

        # Retrieve class from dataframe searching by the image ID
        label = df['Group'].loc[df['ImageDataID'] == image_id].values[0]

        # Assign a class numerical value depending on the group: 0 (CN) or 1 (AD)
        if label in ['CN']:
            labels.append([0]) 
        elif label in ['AD']:
            labels.append([1])  
        else:
            print(f'ERROR with image ID {image_id}')
    
    return np.array(labels)

In [ ]:
# Load training labels
train_labels = return_labels(train_filenames, df)

In [ ]:
# Load validation labels
val_labels = return_labels(val_filenames, df)

In [ ]:
# Load testing labels
test_labels = return_labels(test_filenames, df)

In [ ]:
# Check size of each dataset
print(f'[+] Training labels shape:', train_labels.shape)
print(f'[+] Validation labels shape:', val_labels.shape)
print(f'[+] Testing labels shape:', test_labels.shape)

[+] Training labels shape: (802, 1)
[+] Validation labels shape: (172, 1)
[+] Testing labels shape: (172, 1)


### Create TFRecords
The following functions have been already defined by Tensorflow, and can be found in this [link](https://www.tensorflow.org/tutorials/load_data/tfrecord).

#### Define functions

In [ ]:
def _bytes_feature(value):
    '''
    Returns a bytes_list from a string / byte.
    '''
    
    if isinstance(value, type(tf.constant(0))): # if value is tensor
        value = value.numpy() # get value of tensor
    
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    '''
    Returns a floast_list from a float / double.
    '''
    
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    '''
    Returns an int64_list from a bool / enum / int / uint.
    '''
    
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_array(array):
    
    array = tf.io.serialize_tensor(array)
    
    return array

def parse_single_volume(volume, label):
    
    # Get first value of label, as it is an array of length 1
    label = label[0]
    
    # Define the dictionary -- the structure -- of our single example
    data = {'height' : _int64_feature(volume.shape[0]),
            'width' : _int64_feature(volume.shape[1]),
            'depth' : _int64_feature(volume.shape[2]),
            'raw_image' : _bytes_feature(serialize_array(volume)),
            'label' : _int64_feature(label)}
    
    # Create an Example, wrapping the single features
    out = tf.train.Example(features = tf.train.Features(feature = data))

    return out

def write_images_to_tfr(volumes, labels, filename = 'images', max_files = 10, out_dir = './Datasets/TFRecords/'):

    # Determine the number of TFRecords needed
    splits = (len(volumes)//max_files) + 1 
    if len(volumes) % max_files == 0:
        splits-=1   
    
    print(f'[+] Number of TFRecords needed for {len(volumes)} volumes: {splits}')
    print(f'    [-] Number of files per TFRecord: {max_files}')
    
    # Check if output directory exists
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)   
    print(f'\n[+] Output directory: {out_dir}\n')
    
    # Write TFRecords
    file_count = 0
    
    for i in tqdm.tqdm(range(splits)):
        
        # Retrieve name of the TFRecord
        tfr_name = '{}{}_{}.tfrecords'.format(out_dir, i+1, filename)
        print(f'[+] Writing TFRecord: {tfr_name}')

        # Start writer
        writer = tf.io.TFRecordWriter(tfr_name)
        current_tfr_count = 0
    
        while current_tfr_count < max_files: 
            
            # Get the index of the file that we want to parse now
            index = i * max_files + current_tfr_count
            
            # Check if all dataset has been added to TFRecords
            if index == len(volumes):
                break
                
            # Retrieve volume and label
            current_volume = volumes[index]
            current_label = labels[index]

            # Create the required example representation
            out = parse_single_volume(volume = current_volume, label = current_label)

            writer.write(out.SerializeToString())
            
            # Update counters
            current_tfr_count+=1
            file_count += 1
       
        # Close writer
        writer.close()
    
    print(f'Number of files wrote to TFRecords: {file_count}')

#### Create training TFRecords

In [ ]:
write_images_to_tfr(train_dataset, train_labels, 
                    max_files = 30, 
                    filename = 'train_volumes',
                    out_dir = './Datasets/TFRecords/Train/')

[+] Number of TFRecords needed for 802 volumes: 27
    [-] Number of files per TFRecord: 30

[+] Output directory: ../Datasets/TFRecords/Train/



  0%|          | 0/27 [00:00<?, ?it/s]

[+] Writing TFRecord: ../Datasets/TFRecords/Train/1_train_volumes.tfrecords


  4%|▎         | 1/27 [00:01<00:40,  1.57s/it]

[+] Writing TFRecord: ../Datasets/TFRecords/Train/2_train_volumes.tfrecords


...

[+] Writing TFRecord: ../Datasets/TFRecords/Train/26_train_volumes.tfrecords


 96%|█████████▋| 26/27 [00:49<00:01,  1.58s/it]

[+] Writing TFRecord: ../Datasets/TFRecords/Train/27_train_volumes.tfrecords


100%|██████████| 27/27 [00:50<00:00,  1.87s/it]

Number of files wrote to TFRecords: 802


#### Create validation TFRecords

In [ ]:
write_images_to_tfr(val_dataset, val_labels, 
                    max_files = 30, 
                    filename = 'val_volumes',
                    out_dir = './Datasets/TFRecords/Validation/')

  0%|          | 0/6 [00:00<?, ?it/s]

[+] Number of TFRecords needed for 172 volumes: 6
    [-] Number of files per TFRecord: 30

[+] Output directory: ../Datasets/TFRecords/Validation/

[+] Writing TFRecord: ../Datasets/TFRecords/Validation/1_val_volumes.tfrecords


 17%|█▋        | 1/6 [00:01<00:06,  1.24s/it]

[+] Writing TFRecord: ../Datasets/TFRecords/Validation/2_val_volumes.tfrecords


 33%|███▎      | 2/6 [00:02<00:05,  1.40s/it]

...

[+] Writing TFRecord: ../Datasets/TFRecords/Validation/5_val_volumes.tfrecords


 83%|████████▎ | 5/6 [00:07<00:01,  1.55s/it]

[+] Writing TFRecord: ../Datasets/TFRecords/Validation/6_val_volumes.tfrecords


100%|██████████| 6/6 [00:08<00:00,  1.50s/it]

Number of files wrote to TFRecords: 172


#### Create testing TFRecords

In [ ]:
write_images_to_tfr(test_dataset, test_labels, 
                    max_files = 30, 
                    filename = 'test_volumes',
                    out_dir = './Datasets/TFRecords/Test/')

  0%|          | 0/6 [00:00<?, ?it/s]

[+] Number of TFRecords needed for 172 volumes: 6
    [-] Number of files per TFRecord: 30

[+] Output directory: ../Datasets/TFRecords/Test/

[+] Writing TFRecord: ../Datasets/TFRecords/Test/1_test_volumes.tfrecords


 17%|█▋        | 1/6 [00:01<00:07,  1.47s/it]

[+] Writing TFRecord: ../Datasets/TFRecords/Test/2_test_volumes.tfrecords


 33%|███▎      | 2/6 [00:03<00:06,  1.52s/it]

...

[+] Writing TFRecord: ../Datasets/TFRecords/Test/5_test_volumes.tfrecords


 83%|████████▎ | 5/6 [00:08<00:01,  1.84s/it]

[+] Writing TFRecord: ../Datasets/TFRecords/Test/6_test_volumes.tfrecords


100%|██████████| 6/6 [00:09<00:00,  1.58s/it]

Number of files wrote to TFRecords: 172
